# **MOVIE PLOTS CLASSIFICATION PROJECT**
Students:
*Edoardo Frigerio, Mattia De Masi, Lorenzo Scandolara*
Professor: Andrea Belli
Subject: Text Mining

**Introduction**

This project is based on a dataset freely available on github at https://media.githubusercontent.com/media/nluninja/nlp_datasets/main/wikipedia_movie_plots/data/wiki_movie_plots_deduped.csv .

The dataset is composed by 34886 observations gathered from 1901 to 2017. For each observation we have some relevant features such as the title, year release, genre and the plot from wikipedia.

Our aim is to train a model in order to be able to classify plots of movies with different labels corresponding to different movies genre. We have used 2 different architecture (***LSTM*** and ***BERT***) and we have tried to compare these models to understand strengths and weaknesses for each.




## (1) Import libraries
This cell is used to import all the necessary packages and libraries that we have used in the project.

In [ ]:
!pip install transformers
!pip install --upgrade tqdm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from collections import Counter
import nltk
import spacy 
import seaborn as sns
from tqdm import tqdm
from sklearn import metrics
from nltk.corpus import stopwords
nlp = spacy.load('en_core_web_sm')
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, TimeDistributed, Dropout, Bidirectional, InputLayer, concatenate, Reshape
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import torch
import re
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.datasets import make_classification
import string
import itertools
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("wordnet")

## (2) Load the Data
Our dataset is made up of 34886 rows and 8 columns. The columns represent:
* Release Year
* Title
* Origin/Ethnicity
* Director
* Cast
* Genre
* Wiki Page
* Plot

Our aim is find the movie's Genre starting from the description in the Plot column.



In [ ]:
movies = pd.read_csv('https://media.githubusercontent.com/media/nluninja/nlp_datasets/main/wikipedia_movie_plots/data/wiki_movie_plots_deduped.csv')

In [ ]:
movies.head()

In [ ]:
len(movies)

## (3) Data cleaning
Let's start to clean the dataset. 
We are going to remove all columns which are not useful for our task. We end up with a dataset composed only with "Genre", "Release Year" and "Plot" colums form the previous dataset.

In [ ]:
movies_edit = movies[["Genre","Release Year","Plot"]]

Let's remove nuisances as unknown values or duplicates.

In [ ]:
i = movies_edit[(movies_edit.Genre == 'unknown')].index
movies_edit=movies_edit.drop(i)
i = movies_edit[(movies_edit.Genre ==' ') ].index
movies_edit=movies_edit.drop(i)
movies_edit=movies_edit.drop_duplicates(subset=['Plot'])
movies_edit=movies_edit.reset_index()

In [ ]:
movies_edit.head()

We dropped 6561 observations from the dataset

In [ ]:
len(movies) - len(movies_edit)

### (3.1) Aggregation of classes


'Genre' is a general classification with possibly more than one label for each observation, we want to reduce the 'Genre' column so that we have only one single label for each observation. To do so we will add a new column to the dataset called 'First_genre', as we are assuming that the first reported genre is the most relevant for each movie.


In [ ]:
First_Genre=[]
for i in range(len(movies_edit)):
    split_genre=movies_edit.loc[i,'Genre'].split(",")
    First_Genre.append(split_genre[0])

In [ ]:
for i in range(len(First_Genre)):
    split_genre=First_Genre[i].split("/")
    First_Genre[i] = split_genre[0]

Our aim is to create 5 macro-classes with the biggest possible number of observation each. In the dataset we have some labels for the 'Genre' column that we want to aggregate with the most popular classes that we will lately use in the models training. The cell below shows the aggregation process.

In [ ]:
for i in range(len(First_Genre)):
  if First_Genre[i] == "action ":
     First_Genre[i] = ("action")
  elif First_Genre[i] == "world war ii" or First_Genre[i] == "war":
    First_Genre[i] = ("action")
  elif First_Genre[i] == "crime":
    First_Genre[i] = ("thriller")

Then we finally add this new column to the dataset.

In [ ]:
movies_edit["First_genre"] = First_Genre

In [ ]:
movies_edit['First_genre'].value_counts().head(5)

In [ ]:
labels_perc = movies_edit['First_genre'].value_counts()/len(movies_edit['First_genre'])

In [ ]:
labels_perc.head(5)

We have almost **60%** of observation within the first 5 classes

In [ ]:
labels_perc[0:5].sum()

Displaying frequency for each **relevant** class.

In [ ]:
labels_perc[:5].plot.bar()
plt.xlabel('Genre')
plt.ylabel('Relative Frequency')
plt.title('Bar chart for classes')
plt.show()

Now we create a new dataset composed only by the 5 classes mentioned above

In [ ]:
movies_edit2 = movies_edit.groupby("First_genre").filter( lambda x: len(x)>=1200)

In [ ]:
movies_edit2['First_genre'].value_counts()/len(movies_edit2)

It is clear that this dataset is **not balanced**. We will asses this balancing problem later in the notebook.

## (4) Lemmatization 
We define a function to lemmatize words. 

In [ ]:
movies_edit2['clean_plot']=movies_edit2['Plot'].str.lower()

In [ ]:
def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)

In [ ]:
movies_edit2['clean_plot'] = list(map(getLemmText,movies_edit2['clean_plot']))

**Stop words**

The package nltk provide us a list of the most common words in english. We decide to add in this list some words that we've noticed are recurrent in all the plot and so are not useful for classification: 
* film
* movie
* go
* one
* two
* take
* get
* find
* back
* tell
* ha
* wa
* and other unuseful words

Our aim is remove all the stop words. 

In [ ]:
movies_edit2 = movies_edit2.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;.]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = stopwords.words('english')
STOPWORDS.append('film')
STOPWORDS.append('movie')
STOPWORDS.append('one')
STOPWORDS.append('two')
STOPWORDS.append('get')
STOPWORDS.append('back')
STOPWORDS.append('tell')
STOPWORDS.append('new')
STOPWORDS.append('also')
STOPWORDS.append('time')
STOPWORDS.append('hi')
STOPWORDS.append('ha')
STOPWORDS.append('wa')
STOPWORD = set(STOPWORDS)

In [ ]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [ ]:
movies_edit2['clean_plot'] = movies_edit2['clean_plot'].apply(clean_text)
movies_edit2['clean_plot'] = movies_edit2['clean_plot'].str.replace('\d+', '')

We want to understand what are the most common stop words, we use a bar chart in order to visualize it

In [ ]:
#Counting and displaying the occurrences of a list of stopwords
nlplist = []
for i in range(len(movies_edit2['Plot'])):
    nlplist.append(str(movies_edit2['Plot'][i]))
listToStr = ' '.join([str(elem) for elem in nlplist])

Dict_stop = {}
for word in STOPWORD:
     Dict_stop[word] = listToStr.count(word)

top30 =sorted(Dict_stop.items(), key=lambda x:x[1],reverse=True)[:30]


In [ ]:
plt.figure(figsize=(20,10))
x,y=zip(*top30) # x is the tuple containing the 30 most common words, y is a tuple containing counts
plt.ylabel('Count')
plt.xlabel('Words')
plt.title('30 most common Stopwords')
plt.bar(x,y)
plt.show()

Counting the stopwords for each Genre
 



In [ ]:
def count_stopwords(x):
    n_stopwords = 0

    for word in str(x).split(' '):
        if word in STOPWORDS:
            n_stopwords += 1

    return n_stopwords

We show how many stopwords in average per Genre are inside the 'clean_plot'.

In [ ]:
movies_edit2["stopwords"] = movies_edit2["Plot"].apply(count_stopwords)

In [ ]:
stopwords_label = movies_edit2.groupby(by='First_genre')['stopwords'].agg('mean')
stopwords_label.plot.bar(title = "mean stopwords per genre")
plt.show()

Now we define the function to remove all the stopwords.

In [ ]:
movies_edit2.drop('index', axis = 1)

In the two line below we can see the differences between the starting plot and the final clean_plot.

In [ ]:
print(movies_edit2.loc[1]["Plot"])

In [ ]:
print(movies_edit2.loc[1]["clean_plot"])

In [ ]:
list_plot = movies_edit2["clean_plot"].to_list()
len_plot = [len(str(plot).split(" ")) for plot in list_plot]

In [ ]:
movies_edit2['clean_plot'] = movies_edit2['clean_plot'].apply(lambda x: str(x))

As we said before we have to balance the dataset. We have decided to undersample the dataset. Doing so we have randomly taken 1260 observations from each macro-class.

In [ ]:
movies_edit3=pd.concat([movies_edit2.loc[movies_edit2["First_genre"]=="comedy"].sample(n=1260,replace=False,random_state=1200),
                        movies_edit2.loc[movies_edit2["First_genre"]=="drama"].sample(n=1260,replace=False,random_state=1200),
                        movies_edit2.loc[movies_edit2["First_genre"]=="horror"],
                        movies_edit2.loc[movies_edit2["First_genre"]=="thriller"].sample(n=1260,replace=False,random_state=1200),
                        movies_edit2.loc[movies_edit2["First_genre"]=="action"].sample(n=1260,replace=False,random_state=1200)])

In [ ]:
movies_edit3['First_genre'].value_counts()/len(movies_edit3)

## (5) Graphical overview


#### (5.1) **Number of movies per 5 years span**

In [ ]:
plt.figure(figsize=(10,5));

movies_edit3["Release Year"].plot.hist(stacked=True, bins=20)
plt.xlabel('YEARS')
plt.ylabel('# MOVIES')
plt.title('NUMBER OF MOVIES PER 5 YEARS SPAN')
plt.show()

#### (5.2) **Number of words in each plot**
The graph is in range (0,2000) to make it more readable but there are some plots with more than 4000 words.

In [ ]:
list_plot = movies_edit3["Plot"].to_list()
len_plot = [len(str(plot).split(" ")) for plot in list_plot]
movies_edit3["len_plot"] = len_plot

In [ ]:
plt.figure(figsize=(10,5));
movies_edit3["len_plot"].plot.hist(stacked=True, bins=20, range=(0,2000))
plt.xlabel('NUMBER OF WORDS')
plt.ylabel('NUMBER OF PLOT')
plt.title('NUMBER OF WORDS IN EACH PLOT')
plt.show()

#### (5.3) **Number of words per category** 

In [ ]:
plt.figure(figsize=(10,5));
len_plot_label = movies_edit3.groupby(by='First_genre')['len_plot'].agg('mean')
len_plot_label.plot.bar(title = "WORDS PER LABEL (MEAN)", ylabel = '# OF WORDS', xlabel = 'GENRE')
plt.show()

#### (5.4) **Number of stopwords per Genre**

In [ ]:
plt.figure(figsize=(10,5));
stopwords_label = movies_edit3.groupby(by='First_genre')['stopwords'].agg('mean')
stopwords_label.plot.bar(title = 'STOPWORDS PER GENRE')
plt.xlabel('FIRST GENRE')
plt.show()

#### (5.5) **Percentage of stopwords per Genre**

In [ ]:
movies_edit3["perc_stopwords"] = movies_edit3["stopwords"]/movies_edit3["len_plot"]

plt.figure(figsize=(10,5));
stopwords_perc = movies_edit3.groupby(by='First_genre')['perc_stopwords'].agg('mean')
stopwords_perc.plot.bar(title = "% OF STOPWORDS PER LABEL")
plt.xlabel('FIRST GENRE')
plt.show()

#### (5.6) **Most used word**

In [ ]:
movies_edit3 = movies_edit3.reset_index(drop=True)

In [ ]:
#which words are the most frequent in the dataset without stopwords
movies_stop = movies_edit3[['First_genre','clean_plot']].copy() #create a copy of df and removing stopwords
STOPWORDS = set(nlp.Defaults.stop_words)

movies_stop['PLOT'] = movies_stop['clean_plot'].apply(lambda x:str(x).split())

freq_words = Counter([word for text in movies_stop['PLOT'] for word in text if len(word)>2])
freq_word_df = pd.DataFrame(freq_words.most_common(150), columns=['Word','Count'])
plt.figure(figsize=(20,10));
sns.barplot(y='Word',x='Count',data=freq_word_df[:25], color= 'tab:blue').set_title("25 MOST FREQUENT WORDS IN TEXT")
plt.xlabel('COUNT')
plt.ylabel('WORD')

plt.show()

In [ ]:
most_frequent = movies_edit3.groupby("First_genre")["clean_plot"].apply(lambda x: Counter(" ".join(x).split()).most_common(20))

In [ ]:
genre = sorted(movies_edit3['First_genre'].unique())
for i in range(len(genre)):
  q=[]
  print(genre[i],":")
  for j in range(20):
    q.append(most_frequent[i][j][0])
  print(q)

## (6) Recurrent Neural Network

### (6.1) LSTM

Defining parameters

In [ ]:
MAX_NB_WORDS = 6000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(movies_edit3['clean_plot'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(movies_edit3['clean_plot'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(movies_edit3['First_genre']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Defining model architecture:

- Embedding layer

- Dropout layer ( SpatialDropout1D will help promote independence between feature map)

- LSTM layer

- Dense layer

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 6
batch_size = 64

#### (6.1.1) Training the model

In [ ]:
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.legend()
plt.show()

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.legend()
plt.show()

In [ ]:
y_predict = np.argmax(model.predict(X_test), axis=-1)
y_true = Y_test.argmax(axis=1)
label_names = movies_edit3['First_genre'].unique()

In [ ]:
print(classification_report(y_true, y_predict, target_names=label_names))

#### (6.1.2) Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true,y_predict)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = label_names)
disp.plot()
plt.show()

### (6.2) Bi-LSTM

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 6
batch_size = 64

#### (6.2.1) Training the model

In [ ]:
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.legend()
plt.show()

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.legend()
plt.show()

In [ ]:
y_predict = np.argmax(model.predict(X_test), axis=-1)
y_true = Y_test.argmax(axis=1)
label_names = movies_edit3['First_genre'].unique()

In [ ]:
print(classification_report(y_true, y_predict, target_names=label_names))

#### (6.2.2)Confusion matrix

In [ ]:
cm = confusion_matrix(y_true,y_predict)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = label_names)
disp.plot()
plt.show()

## (7) Bert

### (7.1) Preprocessing data
We create a new dataframe with only two columns: 
* clean_plot (our independent variable)
* First_Genre (our response)


In [ ]:
df=movies_edit3[['clean_plot','First_genre']]

In [ ]:
df =df.rename(columns={'clean_plot':'TITLE','First_genre':'CATEGORY'})

We define a new function to factorize each genre from 0 to 5.

In [ ]:
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

Defining some key variables that will be used later on in the training

In [ ]:
MAX_LEN = 100
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2 
EPOCHS = 6
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.TITLE[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

Creating the dataset and dataloader for the neural network

In [ ]:
train_size = 0.6
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### (7.2) Bert Model

Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

In [ ]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = DistillBERTClass()
model.to(device)

Creating the loss function and optimizer

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

Function to calcuate the accuracy of the model

In [ ]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

Defining the training function on the 80% of the dataset for tuning the distilbert model

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%100==0:
          loss_step = tr_loss/nb_tr_steps
          accu_step = (n_correct*100)/nb_tr_examples 
          print(f"Training Accuracy per step {nb_tr_steps}: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

#### (7.2.1) Training the model

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

#### (7.2.2) Testing the model

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    nb_tr_steps=0 ;nb_tr_examples=0 ; tr_loss=0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

#### (7.2.3) Accuracy report


In [ ]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

## (8) Conclusion

We work on three different architectures: 
1. LSTM
2. Bi-LSTM
3. BERT 

The first two architecture gave us very similar output but Bi-LSTM performed better, indeed we reach an accuracy of 52%, 4% better than the LSTM. 

Even after dropping the most recurrent words each plot has similar text. Moreover there isn't a common link between the same genre

As comparison we have used accuracy for all 3 models.

BERT gave us a slightly worse result: only 42,54% of test accuracy. The problem could be discernible from the fact that we have used only 6 epochs to have a comparison between models and BERT's response is less functional than the response of other two models. 


All the models that we reported here don't have an accuracy as high as in other examples of text classification. This is due to the fact that the original classification in the dataset is not as specific as it should be. A lot of movies have more than 1 genre and the classification is very broad in terms of different genre taken into consideration. Moreover the genre that we choose are slightly similar (i.e. for the thriller, action and horror  genres the difference in the plots is not as marked as it should be in order to train a well-prepared model for classification) in terms of plots. 

Summing up we are happy with the results that we have obtained and we think this has been a valuable exercise for our future experiences.